In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve,convolve2d
import math
from PIL import Image
import numpy as np 
import copy
from collections import deque


In [2]:
def image_registration(img1, img2):
    # Open the image files.
    img1_color = cv2.imread(img1)  # Image to be aligned.
    img2_color = cv2.imread(img2)    # Reference image.
 
    # Convert to grayscale.
    img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
    height, width = img2.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not required in this case).
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)

    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)

    # Sort matches on the basis of their Hamming distance.
    matches.sorted(key = lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
      p1[i, :] = kp1[matches[i].queryIdx].pt
      p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv2.warpPerspective(img1_color,
                        homography, (width, height))

    # Plot the output images.
    plt.axis('off')
    plt.imshow(transformed_img,cmap='gray', vmin=0, vmax=255)
    plt.show()

In [3]:
#For computing motion tracking function for two images
def motionTrackingFunction(img1,img2):
    Iref=img1
    Inext=img2
    Iref=np.array(Iref).astype(np.float32)
    Inext=np.array(Inext).astype(np.float32)
    kernel_x = np.array([[-1., 1.], [-1., 1.]])*.25
    kernel_y = np.array([[-1., -1.], [1., 1.]])*.25
    kernel_t = np.array([[1., 1.], [1., 1.]])*.25
    Iref = Iref / 255. 
    Inext = Inext / 255. 
    Ix=cv2.filter2D(Iref,-1,kernel=kernel_x)
    Iy=cv2.filter2D(Iref,-1,kernel=kernel_y)
    It=cv2.filter2D(Iref,-1,kernel=kernel_t)+cv2.filter2D(Inext,-1,kernel=kernel_x)
    Ix,Iy,It=np.array(Ix),np.array(Iy),np.array(It)
    u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
    return u

for i in range(0,240,30):
    Iref=cv2.imread(r'I:\murali\Assignment3\question1\imagesFromVideo\frame%d.jpg'%i,cv2.IMREAD_GRAYSCALE)
    Inext=cv2.imread(r'I:\murali\Assignment3\question1\imagesFromVideo\frame%d.jpg'%(i+30),cv2.IMREAD_GRAYSCALE)
    print("Motion function estamite for frame" + str(i) + " frame" + str(i+30) + str(motionTrackingFunction(Iref,Inext)))

C:\Users\NITINK~1\AppData\Local\Temp/ipykernel_10596/2521721863.py:16: RuntimeWarning: divide by zero encountered in true_divide
  u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
C:\Users\NITINK~1\AppData\Local\Temp/ipykernel_10596/2521721863.py:16: RuntimeWarning: invalid value encountered in true_divide
  u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))


Motion function estamite for frame0 frame30[[      nan       nan       inf ... 329.50003       inf       inf]
 [      nan       nan       inf ... 329.50003       inf       inf]
 [      nan       nan       inf ... 465.27274 465.98343       inf]
 ...
 [      inf       inf       inf ... 169.5     167.5     234.75946]
 [      inf       inf       inf ... 239.70921 167.00002 165.5    ]
 [      inf       inf       inf ... 238.29501 117.7333  232.63814]]
Motion function estamite for frame30 frame60[[       inf        inf  37.000004 ... 630.0322          inf 447.      ]
 [       inf        inf  37.000004 ... 630.03705         inf 447.      ]
 [ 37.000004  37.000004  53.033016 ... 630.0322          inf 447.      ]
 ...
 [       inf        inf        inf ... 381.1306   382.54187  384.66617 ]
 [ 34.648235  34.648235  25.000002 ... 270.00003         inf 384.6632  ]
 [ 35.355343  36.769554  37.47666  ... 190.21248  381.1306          inf]]
Motion function estamite for frame60 frame90[[       inf     